In [1]:
import pandas as pd
import math
import collections
import datetime


# 국가, 날짜별 치사율 데이터 가져오기

#### 데이터 소스: https://github.com/owid/covid-19-data/tree/master/public/data

In [2]:
data = pd.read_csv("../../data/owid-covid-data.csv")
data

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17420,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17421,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17422,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17423,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 국가 filtering하기

In [3]:
countrySeries = data.iso_code
countryList = list(set(countrySeries)) # set으로 만들어서 중복 다 없앰.

len(countryList)

212

In [4]:

caseData = data.total_cases

In [5]:
selectedCountries = []
countryRowList = []
for country in countryList:
    check = False
    for i in range(len(countrySeries)):
        if(country == countrySeries[i]):
            check = True
        if(check and country != countrySeries[i]): # 맨 마지막 total값을 가져오기 위해서 country[i]이름이 달라지면 i-1을 리스트에 넣는다.
            if(caseData[i-1] > 3000 and country != 'OWID_WRL'): # case가 3000명이 넘는 것만 들고 옴, OWID_WRL은 전세계 데이터라 뺌
                countryRowList.append(i-1)
                selectedCountries.append(country)
            break
            
len(countryRowList) 

68

In [6]:
selectedCountries

['GBR',
 'MDA',
 'MYS',
 'GHA',
 'KWT',
 'DZA',
 'AUT',
 'AUS',
 'PER',
 'ITA',
 'PAK',
 'POL',
 'MEX',
 'BRA',
 'JPN',
 'CAN',
 'BGD',
 'PAN',
 'ZAF',
 'ARE',
 'ESP',
 'COL',
 'MAR',
 'CZE',
 'IRL',
 'CHN',
 'LUX',
 'DNK',
 'RUS',
 'NGA',
 'KAZ',
 'KOR',
 'DEU',
 'ROU',
 'FIN',
 'BLR',
 'FRA',
 'IRQ',
 'IND',
 'USA',
 'SRB',
 'PRT',
 'IRN',
 'CHE',
 'ECU',
 'THA',
 'AFG',
 'ISR',
 'BEL',
 'BOL',
 'NOR',
 'NLD',
 'EGY',
 'DOM',
 'PHL',
 'BHR',
 'IDN',
 'HUN',
 'QAT',
 'OMN',
 'ARM',
 'SWE',
 'CHL',
 'SAU',
 'SGP',
 'TUR',
 'ARG',
 'UKR']

### 일별 치사율 구하기

In [7]:

data = data[["iso_code",'date', 'new_cases', 'new_deaths']] 
fatalityDataList = data.values.tolist()

In [8]:
fatalityDataList

[['ABW', '2020-03-13', 2, 0],
 ['ABW', '2020-03-20', 2, 0],
 ['ABW', '2020-03-24', 8, 0],
 ['ABW', '2020-03-25', 5, 0],
 ['ABW', '2020-03-26', 2, 0],
 ['ABW', '2020-03-27', 9, 0],
 ['ABW', '2020-03-28', 0, 0],
 ['ABW', '2020-03-29', 0, 0],
 ['ABW', '2020-03-30', 22, 0],
 ['ABW', '2020-04-01', 5, 0],
 ['ABW', '2020-04-02', 0, 0],
 ['ABW', '2020-04-03', 5, 0],
 ['ABW', '2020-04-04', 2, 0],
 ['ABW', '2020-04-05', 2, 0],
 ['ABW', '2020-04-06', 0, 0],
 ['ABW', '2020-04-07', 7, 0],
 ['ABW', '2020-04-08', 3, 0],
 ['ABW', '2020-04-09', 3, 0],
 ['ABW', '2020-04-10', 5, 0],
 ['ABW', '2020-04-11', 4, 0],
 ['ABW', '2020-04-12', 6, 0],
 ['ABW', '2020-04-13', 0, 0],
 ['ABW', '2020-04-14', 0, 0],
 ['ABW', '2020-04-15', 0, 0],
 ['ABW', '2020-04-16', 1, 1],
 ['ABW', '2020-04-17', 2, 0],
 ['ABW', '2020-04-18', 1, 1],
 ['ABW', '2020-04-19', 0, 0],
 ['ABW', '2020-04-20', 1, 0],
 ['ABW', '2020-04-21', 0, 0],
 ['ABW', '2020-04-22', 0, 0],
 ['ABW', '2020-04-23', 3, 0],
 ['ABW', '2020-04-24', 0, 0],
 ['ABW', 

# Factor 데이터 time-series로 바꾸기

#### 데이터 소스: https://www.kaggle.com/vishalvjoseph/weather-dataset-for-covid19-predictions

In [33]:
data = pd.read_csv("../../data/uv/UV_Processed.csv")
data=data.values.tolist()
len(data)
data

[['AFG', '2019-12-01', 0.0949],
 ['AFG', '2019-12-02', 0.1062],
 ['AFG', '2019-12-03', 0.1079],
 ['AFG', '2019-12-04', 0.1073],
 ['AFG', '2019-12-05', 0.0999],
 ['AFG', '2019-12-06', 0.0927],
 ['AFG', '2019-12-07', 0.0971],
 ['AFG', '2019-12-08', 0.096],
 ['AFG', '2019-12-09', 0.0754],
 ['AFG', '2019-12-10', 0.0698],
 ['AFG', '2019-12-11', 0.0746],
 ['AFG', '2019-12-12', 0.0681],
 ['AFG', '2019-12-13', 0.069],
 ['AFG', '2019-12-14', 0.0754],
 ['AFG', '2019-12-15', 0.0768],
 ['AFG', '2019-12-16', 0.0753],
 ['AFG', '2019-12-17', 0.078],
 ['AFG', '2019-12-18', 0.0797],
 ['AFG', '2019-12-19', 0.0727],
 ['AFG', '2019-12-20', 0.0753],
 ['AFG', '2019-12-21', 0.0771],
 ['AFG', '2019-12-22', 0.0773],
 ['AFG', '2019-12-23', 0.0728],
 ['AFG', '2019-12-24', 0.073],
 ['AFG', '2019-12-25', 0.083],
 ['AFG', '2019-12-26', 0.0875],
 ['AFG', '2019-12-27', 0.0907],
 ['AFG', '2019-12-28', 0.0934],
 ['AFG', '2019-12-29', 0.0823],
 ['AFG', '2019-12-30', 0.0715],
 ['AFG', '2019-12-31', 0.0608],
 ['AFG', '202

In [34]:
newData= []
for row in data:
    if(row[0] in selectedCountries):
        for fRow in fatalityDataList:
            if(fRow[0] == row[0] and fRow[1] == row[1]):
                row.append(fRow[2])
                row.append(fRow[3])
                newData.append(row)
data = newData
len(data)
data

[['AFG', '2019-12-31', 0.0608, 0, 0],
 ['AFG', '2020-01-01', 0.0334, 0, 0],
 ['AFG', '2020-01-02', 0.0548, 0, 0],
 ['AFG', '2020-01-03', 0.0724, 0, 0],
 ['AFG', '2020-01-04', 0.0743, 0, 0],
 ['AFG', '2020-01-05', 0.0595, 0, 0],
 ['AFG', '2020-01-06', 0.0683, 0, 0],
 ['AFG', '2020-01-07', 0.0732, 0, 0],
 ['AFG', '2020-01-08', 0.0768, 0, 0],
 ['AFG', '2020-01-09', 0.0823, 0, 0],
 ['AFG', '2020-01-10', 0.0803, 0, 0],
 ['AFG', '2020-01-11', 0.0757, 0, 0],
 ['AFG', '2020-01-12', 0.07, 0, 0],
 ['AFG', '2020-01-13', 0.0758, 0, 0],
 ['AFG', '2020-01-14', 0.081, 0, 0],
 ['AFG', '2020-01-15', 0.0852, 0, 0],
 ['AFG', '2020-01-16', 0.0827, 0, 0],
 ['AFG', '2020-01-17', 0.0735, 0, 0],
 ['AFG', '2020-01-18', 0.0909, 0, 0],
 ['AFG', '2020-01-19', 0.0786, 0, 0],
 ['AFG', '2020-01-20', 0.0725, 0, 0],
 ['AFG', '2020-01-21', 0.0826, 0, 0],
 ['AFG', '2020-01-22', 0.0911, 0, 0],
 ['AFG', '2020-01-23', 0.099, 0, 0],
 ['AFG', '2020-01-24', 0.0849, 0, 0],
 ['AFG', '2020-01-25', 0.0997, 0, 0],
 ['AFG', '2020-0

### 각 factor 최대 최소 구하기

In [17]:
minValue = [9999999999,9999999999,9999999999,9999999999,9999999999,9999999999]
maxValue = [-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999]

In [18]:
for row in data:
    for j in range(2,3,1):
        if(not math.isnan(row[j])):
            if(float(row[j]) < minValue[j-2]):
                minValue[j-2] = float(row[j])
            if(float(row[j]) >maxValue[j-2]):
                maxValue[j-2] = float(row[j])

In [27]:
print(minValue)
print(maxValue)

[0.0019, 9999999999, 9999999999, 9999999999, 9999999999, 9999999999]
[0.3888, -9999999999, -9999999999, -9999999999, -9999999999, -9999999999]


### factor당 40개로 구간을 나눠 날짜, 국가 채우기

In [28]:
valuePartition = []
for i in range(0,1,1):
    eachValue = []
    for j in range(40):
        partition = minValue[i]+ j*(maxValue[i] - minValue[i])/40.0
        eachValue.append(partition)
    eachValue.append(maxValue[i])
    valuePartition.append(eachValue)

In [29]:
valuePartition

[[0.0019,
  0.0115725,
  0.021244999999999997,
  0.030917499999999994,
  0.040589999999999994,
  0.050262499999999995,
  0.05993499999999999,
  0.0696075,
  0.07927999999999999,
  0.08895249999999999,
  0.09862499999999999,
  0.10829749999999999,
  0.11796999999999998,
  0.1276425,
  0.13731500000000002,
  0.1469875,
  0.15666,
  0.1663325,
  0.176005,
  0.1856775,
  0.19535,
  0.20502249999999997,
  0.214695,
  0.2243675,
  0.23403999999999997,
  0.2437125,
  0.25338499999999997,
  0.2630575,
  0.27273000000000003,
  0.2824025,
  0.292075,
  0.30174749999999995,
  0.31142,
  0.3210925,
  0.330765,
  0.3404375,
  0.35011,
  0.3597825,
  0.369455,
  0.37912749999999995,
  0.3888]]

In [30]:
dataPartition = []
for i in range(0,1,1):
    dataPartition.append([])
    for j in range(40):
        dataPartition[i].append([])

In [31]:
dataPartition

[[[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []]]

In [35]:
for row in data:
    for index in range(2,3,1):
        if(not math.isnan(row[index])):
            for j in range(40):
                if(valuePartition[index-2][j] <= float(row[index])<valuePartition[index-2][j+1]):
                    dataPartition[index-2][j].append([row[0], row[1], row[3], row[4]])
                elif(valuePartition[index-2][40] <= float(row[index])):
                    dataPartition[index-2][39].append([row[0], row[1], row[3], row[4]])

In [36]:
dataPartition[0][1]

[['AUT', '2020-02-04', 0, 0],
 ['AUT', '2020-02-05', 0, 0],
 ['AUT', '2020-02-19', 0, 0],
 ['AUT', '2020-02-24', 0, 0],
 ['BEL', '2020-02-01', 0, 0],
 ['BEL', '2020-02-02', 0, 0],
 ['BEL', '2020-02-03', 0, 0],
 ['BEL', '2020-02-05', 0, 0],
 ['BEL', '2020-02-06', 0, 0],
 ['BEL', '2020-02-13', 0, 0],
 ['BEL', '2020-02-15', 0, 0],
 ['BEL', '2020-02-19', 0, 0],
 ['BEL', '2020-02-20', 0, 0],
 ['BEL', '2020-02-22', 0, 0],
 ['BEL', '2020-02-24', 0, 0],
 ['BEL', '2020-02-26', 0, 0],
 ['BEL', '2020-02-29', 0, 0],
 ['BEL', '2020-03-02', 1, 0],
 ['BEL', '2020-03-05', 10, 0],
 ['BEL', '2020-03-07', 59, 0],
 ['BEL', '2020-03-08', 60, 0],
 ['BEL', '2020-03-10', 39, 0],
 ['BEL', '2020-03-20', 309, 7],
 ['BLR', '2020-02-02', 0, 0],
 ['BLR', '2020-02-03', 0, 0],
 ['BLR', '2020-02-05', 0, 0],
 ['BLR', '2020-02-07', 0, 0],
 ['BLR', '2020-02-11', 0, 0],
 ['BLR', '2020-02-13', 0, 0],
 ['BLR', '2020-02-14', 0, 0],
 ['BLR', '2020-02-16', 0, 0],
 ['BLR', '2020-02-17', 0, 0],
 ['BLR', '2020-02-18', 0, 0],
 ['B

### Corr구하기

In [37]:
# factor correlation data preparing
factorCorrData = []
for i in range(len(valuePartition)):
    corrData = []
    for j in range(40):
        corrData.append((valuePartition[i][j]+ valuePartition[i][j+1])/2.0)
    factorCorrData.append(corrData)

In [38]:
fatallityCorrData = []
for factor in dataPartition:
    corrData = []
    for index in range(40):
        totalDeath = 0
        totalCase = 0
        for row in factor[index]:
            totalCase += row[2]
            totalDeath += row[3]
        if(totalDeath == 0.0):
            corrData.append(0)
        else:
            corrData.append(float(totalCase)/float(totalDeath))
    fatallityCorrData.append(corrData)
        

In [39]:
for i in range(1):
    correlationDayTime = [fatallityCorrData[i], factorCorrData[i]]

    df = pd.DataFrame(correlationDayTime).T
    corr = df.corr(method= 'pearson')
    print(corr[1][0])

-0.2812029425516756


In [40]:
import json

for i in range(1):
    temp = {}
    for j in range(40):
        temp[valuePartition[i][j]]= []
        for row in dataPartition[i][j]:
            if(row[2] != 0):
                temp[valuePartition[i][j]].append({"iso": str(row[0]), "date":str(row[1]), "fatality rate":str(round(float(row[3])/float(row[2]),2))})
            else:
                temp[valuePartition[i][j]].append({"iso": str(row[0]), "date":str(row[1]), "fatality rate":str(0)})
    temp[valuePartition[i][40]]= []
    with open(str(i)+'.json', 'w')as fp:
        json.dump(temp, fp)